In [1]:

import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.linalg import toeplitz
from numpy import fft 
from core.core import *
from util.detector import * 
from util.estimator import * 
from util.modem import * 

import random
import os 

In [2]:

K = 2   # Number of users (transmitter antenna) 
N = 16   # Number of Receiver (Receive antenna)
M = 4   # level of modulation (4-QAM / QPSK)
T = int(1e2) # Total Transmission rounds 
SNR_max = 30


def make_symbol_space(K, M, cp):
    """
    Makes symbol space S = |M|^K 
    :params: 
    :   K (int) : number of users\n
    :   M (int) : level of modulation\n
    :returns:
    :   symbol_space (np.ndarray) : symbol space S=|M|^K
    """
    index_space = np.indices([M for _ in range(K)]).reshape(K, -1).T 
    symbol_space = np.take(cp, index_space)
    return symbol_space 

def comp2re(H, x, z, y):
    """
    Real representation of complex matrix \n
    \t x = [x_r + j x_i] --->  x_re = [x_re x_im] \n
    \t H_re = \n \t \t[H_real, -H_imag \n
        \t H_imag, H_real]\n
    
    """
    x_re = np.hstack((x.real, x.imag))
    z_re = np.hstack((z.real, x.imag))
    y_re = np.hstack((y.real, y.imag)) 
    H_re = np.block([
        [H.real, -H.imag],
        [H.imag, H.real]
    ])
    return H_re, x_re, z_re, y_re 

snr = 5

# simple constellation points (normalized)
constellation_points = np.array([-1-1j, -1+1j, 1-1j, 1+1j])
constellation_points /= np.sqrt(2) 

# make symbol space = |M|^K  in case of using ML detection 
symbol_space = make_symbol_space(K, M, cp=constellation_points) 

x = np.random.randint(0, M, size=K * T).reshape(T, K, 1) 
x = np.take(constellation_points, x)

# Channel 
H = (1 / np.sqrt(2)) * (np.random.randn(T, N, K) + 1j * np.random.randn(T, N, K))
z = (1 / np.sqrt(2)) * (np.random.randn(T, N, 1) + 1j * np.random.randn(T, N, 1))

r = np.matmul(H, x) + (10**(-snr/20)) * z  

# 1-bit ADC 
y = np.sign(r.real) + 1j * np.sign(r.imag) 

H_re, x_re, z_re, y_re = comp2re(H, x, z, y)

In [3]:
ml(y_re, H_re=H_re, symbol_space=symbol_space, snr_dB=snr) 

/Users/sungweon-hong/Projects/comm/util/detector.py:34: RuntimeWarning: divide by zero encountered in log
  log_likelihood = np.sum(np.log(likelihood), axis=2)


array([[[ 0.70710678+0.70710678j],
        [-0.70710678-0.70710678j]],

       [[-0.70710678-0.70710678j],
        [-0.70710678+0.70710678j]],

       [[ 0.70710678+0.70710678j],
        [-0.70710678+0.70710678j]],

       [[-0.70710678-0.70710678j],
        [-0.70710678-0.70710678j]],

       [[ 0.70710678+0.70710678j],
        [-0.70710678+0.70710678j]],

       [[ 0.70710678+0.70710678j],
        [ 0.70710678-0.70710678j]],

       [[-0.70710678+0.70710678j],
        [ 0.70710678+0.70710678j]],

       [[-0.70710678+0.70710678j],
        [ 0.70710678+0.70710678j]],

       [[ 0.70710678+0.70710678j],
        [-0.70710678+0.70710678j]],

       [[-0.70710678-0.70710678j],
        [ 0.70710678-0.70710678j]],

       [[ 0.70710678+0.70710678j],
        [-0.70710678+0.70710678j]],

       [[-0.70710678+0.70710678j],
        [ 0.70710678-0.70710678j]],

       [[-0.70710678+0.70710678j],
        [-0.70710678+0.70710678j]],

       [[-0.70710678+0.70710678j],
        [ 0.70710678+0.707106